# Проект

Задание для проекта

1.	Выгрузить описания открытых вакансий с любой из площадок (hh.ru, rabota.sber.ru, indeed.com) и сохранить их в БД
2.	Предообработать текст описаний вакансий (токенизировать, нормировать)
3.	Взять любой текст, описывающий опыт из резюме, и аналогично предобработать
4.	Посчитать векторное представление описаний вакансий и опыта из резюме
5.	Посчитать косинусную меру сходства для опыта и описаний вакансий
6.	Ранжировать вакансии по мере их сходства с опытом
7.	Вывести Топ-10 наиболее релевантных
8.	Подготовить скрипт принимающий на вход текстовое описание опыта и выводящий Топ-10 наиболее релевантных вакансий из 1000 случайных


In [2]:
# !pip install ipywidgets
# !pip install pymorphy2
# !pip install pymystem3
# nltk.download('stopwords')

In [1]:
import requests
import json
import codecs
from tqdm.auto import tqdm
from collections import defaultdict
import pickle
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display, clear_output

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

from nltk.corpus import stopwords
from nltk import word_tokenize, regexp_tokenize
import nltk

import re
from string import punctuation, printable

from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, Button, Textarea, FileUpload, Layout

%matplotlib inline

In [41]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', None)

In [3]:
stop_russian = stopwords.words('russian')
sber = 3529
num_per_page = 100
moscow = 1
page = 1
url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={moscow}'

In [4]:
num_pages = requests.get(url).json().get('pages')
print(f'Всего страниц с вакансиями: {num_pages}')

Всего страниц с вакансиями: 12


In [5]:
#формируем список всех id вакансий Сбера

all_vacancy_ids= []
for page in tqdm(range(num_pages)):
    url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={moscow}'
    res = requests.get(url)
    vacancies = res.json()
    vacancy_ids = [el.get('id') for el in vacancies.get('items')]
    all_vacancy_ids.extend(vacancy_ids)
print(f'Всего вакансий: {len(all_vacancy_ids)}')

  0%|          | 0/12 [00:00<?, ?it/s]

Всего вакансий: 1103


In [8]:
# Извлекаем описания вакансий

df = pd.DataFrame()
columns = ['id', 'created_at', 'name', 'description', 'key_skills']
test_id = list(pd.Series(all_vacancy_ids).sample(100)) #100 случайных id для тестов
for n in tqdm(all_vacancy_ids):  #all_vacancy_ids
    url_id = f'https://api.hh.ru/vacancies/{n}'
    res = requests.get(url_id).json()
    df_ = pd.json_normalize(res)[columns]
    df = pd.concat([df, df_], axis=0)
    #dict_id_desc[n] = res_id.json().get('description')

# Переведем даты публикаций в datetime
df['created_at'] = df['created_at'].astype('datetime64[ns]').dt.strftime('%Y-%m-%d')
df.reset_index(drop=True)

In [12]:
# сохраним DataFrame, чтобы не скачивать каждый раз
df.to_pickle('all_raw_description.pkl', compression='zip')

In [43]:
# проверка что все нормально извлекается
df = pd.read_pickle('all_raw_description.pkl', compression='zip').reset_index(drop=True)
df['created_at'] = df['created_at'].astype('datetime64[ns]').dt.strftime('%Y-%m-%d')
df.head(3)

,id,created_at,name,description,key_skills
0,55415805,2022-09-07,Менеджер по работе с клиентами,<p>Сбер — лучший работодатель России. У нас более 60 компаний экосистемы и 14 тысяч подразделений по всей стране.</p> <p>Сбер ищет клиентского менеджера для работы в мобильных офисах. Это специали...,[]
1,69355530,2022-09-23,Финансовый консультант,"<p>Сбер — лучший работодатель России. У нас более 60 компаний экосистемы и 14 тысяч подразделений по всей стране.</p> <p>При оформлении карт Сбера через приложение, клиент может выбрать доставку в...",[]
2,45790891,2022-09-23,Менеджер по работе с ключевыми клиентами малого бизнеса,<p>Сбер — лучший работодатель России. У нас более 60 компаний экосистемы и 14 тысяч подразделений по всей стране.</p> <p> </p> <p>Присоединяйся к команде малого бизнеса Сбера! На позиции «Менеджер...,"[{'name': 'Прямые продажи'}, {'name': 'Финансовый анализ'}, {'name': 'Клиентоориентированность'}, {'name': 'Развитие продаж'}, {'name': 'Ведение переговоров'}, {'name': 'Активные продажи'}, {'name..."


In [46]:
# функция очистки текста от html-тегов,
# в итоге оставим только русские и латинские буквы

def clean_description(row):
    return row.str.replace(r'<[^<>]*>', ' ', regex=True)\
                  .replace(r'[^А-Яа-яёЁa-zA-Z]+', ' ', regex=True)
                    
# функция лемматизации (для токенизации - nltk.regexp_tokenize) -
# длительность лемматизации примерно 45секунд
def tokenize_n_normalize(row, pat=r"\b\w\w+\b", morph=MorphAnalyzer()):
    return [morph.parse(tok)[0].normal_form 
            for tok in regexp_tokenize(row, pat)]

# очистим и лемматизируем текст описания вакансий
df["clean_desc"] = clean_description(df["description"])
df["clean_norm_desc"] = df["clean_desc"].map(lambda x: " ".join(tokenize_n_normalize(x)))
df.head(3)

,id,created_at,name,description,key_skills,clean_desc,clean_norm_desc
0,55415805,2022-09-07,Менеджер по работе с клиентами,<p>Сбер — лучший работодатель России. У нас более 60 компаний экосистемы и 14 тысяч подразделений по всей стране.</p> <p>Сбер ищет клиентского менеджера для работы в мобильных офисах. Это специали...,[],Сбер лучший работодатель России У нас более компаний экосистемы и тысяч подразделений по всей стране Сбер ищет клиентского менеджера для работы в мобильных офисах Это специалист который консульти...,сбер хороший работодатель россия мы более компания экосистема тысяча подразделение по весь страна сбер искать клиентский менеджер для работа мобильный офис это специалист который консультировать к...
1,69355530,2022-09-23,Финансовый консультант,"<p>Сбер — лучший работодатель России. У нас более 60 компаний экосистемы и 14 тысяч подразделений по всей стране.</p> <p>При оформлении карт Сбера через приложение, клиент может выбрать доставку в...",[],Сбер лучший работодатель России У нас более компаний экосистемы и тысяч подразделений по всей стране При оформлении карт Сбера через приложение клиент может выбрать доставку в удобное место и вре...,сбер хороший работодатель россия мы более компания экосистема тысяча подразделение по весь страна при оформление карта сбер через приложение клиент мочь выбрать доставка удобный место время финанс...
2,45790891,2022-09-23,Менеджер по работе с ключевыми клиентами малого бизнеса,<p>Сбер — лучший работодатель России. У нас более 60 компаний экосистемы и 14 тысяч подразделений по всей стране.</p> <p> </p> <p>Присоединяйся к команде малого бизнеса Сбера! На позиции «Менеджер...,"[{'name': 'Прямые продажи'}, {'name': 'Финансовый анализ'}, {'name': 'Клиентоориентированность'}, {'name': 'Развитие продаж'}, {'name': 'Ведение переговоров'}, {'name': 'Активные продажи'}, {'name...",Сбер лучший работодатель России У нас более компаний экосистемы и тысяч подразделений по всей стране Присоединяйся к команде малого бизнеса Сбера На позиции Менеджера по работе с ключевыми клиент...,сбер хороший работодатель россия мы более компания экосистема тысяча подразделение по весь страна присоединяться команда малое бизнес сбер на позиция менеджер по работа ключевой клиент ты быть общ...


In [47]:
# создание виджета для загрузки текствого файла описания резюме
uploader = FileUpload(accept='.txt', multiple=False)
display(uploader)

# Создаем кнопку с нужными параметрами
button_find = Button(description="Найти релевантные резюме", 
                     button_style='success',
                     layout=Layout(width='20%', height='30px')
                    )

def on_button_clicked(b): # Описываем обработчик события нажатия на кнопку
    
    #извлекаем текст резюме из загруженного файла
    for uploaded_filename in uploader.value: 
        resume_title = uploaded_filename
        content = uploader.value[uploaded_filename]['content']   
    resume = codecs.decode(content, encoding="utf-8")  

    # очистим и лемматизируем текст описания резюме
    df_resume = pd.DataFrame([resume], columns=['text_resume'])
    df_resume['clean_text'] = clean_description(df_resume['text_resume'])
    df_resume["clean_norm_text"] = df_resume["clean_text"].map(lambda x: " ".join(tokenize_n_normalize(x)))

    # векторизация описаний вакансий и резюме
    tfidf_vec = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1,1), lowercase=True, max_features=10000)
    X_desc = tfidf_vec.fit_transform(df['clean_norm_desc'])
    X_resume = tfidf_vec.transform(df_resume['clean_norm_text'])

    # поиск косинусной меры сходства и 10 наиболее похожих вакансий
    cos_sim = cosine_similarity(X_resume, X_desc).flatten()
    df_cossim = pd.DataFrame(cos_sim, columns=['cos_sim'])
    index_sim10 = df_cossim.nlargest(10, ["cos_sim"]).index
    df_vacancies_for_resume = pd.merge(df, df_cossim, left_index=True, right_index=True)
    
    
    # распечатка результатов
    print(f'Вакансии для резюме: {resume_title.split(sep=".")[0]}')
    print(f'Токены описания резюме: {"_".join(list(df_resume["clean_norm_text"]))}')
    print('\n10 вакансий с наибольшей косинусной мерой сходства:')
    display(df_vacancies_for_resume[['id', "cos_sim", 'created_at', 'name', "clean_norm_desc"]].iloc[index_sim10])

df_vacancies_for_resume = button_find.on_click(on_button_clicked) # Назначаем обработчик на событие "on_click"
display(button_find) # Отображаем кнопку

FileUpload(value={}, accept='.txt', description='Upload')

Button(button_style='success', description='Найти релевантные резюме', layout=Layout(height='30px', width='20%…

Вакансии для резюме: Аналитик
Токены описания резюме: разработка регулярный отчётность автоматизация операционный процесс написание редактирование скрипт на python парсинга сайт дашбордовый создание нуль дашбордовый power bi подготовка оперативный управленческий отчётност ad hoc задача прогнозирование драйвер канал продажа для цель планирование arima gls февраль март год месяц аналитический центр при правительство российский федерация советник управление конкурентный политика методология закупка анализ больший массив данные tidyverse ggplot etc python pandas numpy seaborn визуализация написание техзадание на дашборд методологический часть анализ подготовка отчёт ход исполнение стратегический документ достижение цель оценка предложение координационный сутевой поддержка куратор проект быстрый аналитика оценка корректность методология оценка позиция подготовка защита аналитический материал по тематика конкурентный политика закупка оценка регулировать воздействие цифра факторный анализ рек

,id,cos_sim,created_at,name,clean_norm_desc
237,69468631,0.181275,2022-09-05,Главный экономист Управления планирования и финансового анализа,обязанность рамка процесс бизнес планирование участие подготовка сводный финансовый модель группа сбер пао сбербанк территориальный банк дочерний компания проведение анализ модель планирование биз...
58,69664316,0.161497,2022-09-09,Аналитик данных/Разработчик отчетности,мы создавать централизовать подразделение разметка данные для машинный обучение наш задача создать качественный сервис для внутренний заказчик по получение разметить датасет обязанность разработка...
59,70194502,0.160474,2022-09-23,Data Analyst / Data Scientist в Сеть продаж,дивизион управление сеть устройство самообслуживание заниматься развитие бизнес банкомат информационно платёжный терминал сфера влияние наш команда задача связанный оптимизация банкоматный сеть ра...
474,69820986,0.152699,2022-09-14,Эксперт (по направлению торговый эквайринг),центр сопровождение карточный продукт клиентский данные подразделение центральный подчинение операционный центр требоваться эксперт по направление торговый эквайринг обязанность проведение работа ...
323,69870384,0.146026,2022-09-15,Data Analyst (Клиентская аналитика),мы искать специалист по работа данные корпоративный блок сбер для аналитический поддержка лидер бизнес подразделение топ менеджмент нужно искать инсайт данные выстраивать система показатель чтобы ...
56,69356019,0.140004,2022-08-31,Финансовый менеджер/аналитик,обязанность участие процесс бизнес планирование pnl cf подготовка анализ управленческий отчётность план факт анализ прогноз исполнение пр подготовка информационно аналитический материал по расход ...
269,70284746,0.139055,2022-09-24,Руководитель проектов разработки и реализации стратегии,департамент стратегия сбер искать руководитель проект разработка реализация стратегия наш команда заниматься создание развитие стратегия сбер мы изучать тренд рынок формировать корректировать стра...
556,69479255,0.136942,2022-09-05,Менеджер по оценке бизнеса и корпоративным финансам,команда корпоративный блок сбер требоваться профессионал область оценка бизнес корпоративный финансы сегмент крупный клиент сделка центральный аппарат связь чем требоваться высокий уровень квалифи...
254,69540885,0.135186,2022-09-06,Методолог,команда сбер поиск методолог на проект инновационный проект новый технология блокчейн сегмент высокодоходный клиент центр корпоративный решение команда заниматься анализ проработка обеспечение гот...
518,69317371,0.130737,2022-08-30,"Эксперт по направлению контроля качества, эффективности и производственной отчетности",центр сопровождение общебанковский операция операционный центр требоваться эксперт по направление контроль качество эффективность отчётность по производственный направление центр обязанность монит...
